In [0]:
df_itensProva_silver = spark.read.table("setoreducacao.bronze.itensProva_2024")
display(df_itensProva_silver)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import(
    col, to_date, year, lit, 
    when, trim, regexp_replace
)
from pyspark.sql.types import(
    IntegerType, DoubleType, StringType, FloatType, DataType
)

In [0]:
#Renomeação de colunas
column_mapping = {
   "CO_POSICAO": "posicao_do_item_na_prova",
   "SG_AREA": "area_de_conhecimento_do_item",
   "CO_ITEM": "codigo_do_item",
   "TX_GABARITO": "gabarito_do_item",
   "CO_HABILIDADE": "habilidade_do_item",
   "IN_ITEM_ABAN": "indicador_de_item_abandonado",
   "TX_MOTIVO_ABAN": "motivo_para_o_abandono_do_item",
   "NU_PARAM_A": "parametro_de_discriminacao",
   "NU_PARAM_B": "parametro_de_dificuldade",
   "NU_PARAM_C": "parametro_de_acerto_ao_acaso",
   "TX_COR": "cor_da_prova",
   "CO_PROVA": "identificador_da_prova",
   "TP_LINGUA": "lingua_estrangeira",
   "IN_ITEM_ADAPTADO": "item_pertencente_a_prova_adaptada" 
}

for old_col, new_col in column_mapping.items():
    df_itensProva_silver = df_itensProva_silver.withColumnRenamed(old_col, new_col)

In [0]:
display(df_itensProva_silver)

In [0]:
from pyspark.sql import functions as F

In [0]:
# Colunas Numéricas (Identificadores e Contagens)
df_itensProva_silver = df_itensProva_silver.withColumn("posicao_do_item_na_prova", F.col("posicao_do_item_na_prova").cast(IntegerType()))
df_itensProva_silver = df_itensProva_silver.withColumn("codigo_do_item", F.col("codigo_do_item").cast(IntegerType()))
df_itensProva_silver = df_itensProva_silver.withColumn("habilidade_do_item", F.col("habilidade_do_item").cast(IntegerType()))
df_itensProva_silver = df_itensProva_silver.withColumn("indicador_de_item_abandonado", F.col("indicador_de_item_abandonado").cast(IntegerType()))
df_itensProva_silver = df_itensProva_silver.withColumn("identificador_da_prova", F.col("identificador_da_prova").cast(IntegerType()))
df_itensProva_silver = df_itensProva_silver.withColumn("item_pertencente_a_prova_adaptada", F.col("item_pertencente_a_prova_adaptada").cast(IntegerType()))

In [0]:
# Colunas Numéricas (precisão decimal)
df_itensProva_silver = df_itensProva_silver.withColumn("parametro_de_discriminacao", F.col("parametro_de_discriminacao").cast(DoubleType()))
df_itensProva_silver = df_itensProva_silver.withColumn("parametro_de_dificuldade",F.col("parametro_de_dificuldade").cast(DoubleType()))
df_itensProva_silver = df_itensProva_silver.withColumn("parametro_de_acerto_ao_acaso",F.col("parametro_de_acerto_ao_acaso").cast(DoubleType()))

In [0]:
# Lista das colunas de parâmetros TRI
cols_tri = [
    "parametro_de_discriminacao",     # A
    "parametro_de_dificuldade",       # B
    "parametro_de_acerto_ao_acaso"    # C
]

# Converter para double e corrigir valores absurdos (>10) dividindo por 1e6
for c in cols_tri:
    df_itensProva_silver = df_itensProva_silver.withColumn(
        c,
        F.when(
            F.abs(F.col(c)) > 10,     # valor fora da realidade da TRI - corrigindo decimal perdido
            F.col(c) / F.lit(1e6)     
        ).otherwise(F.col(c))        
    )


In [0]:
# Colunas Alfanuméricas (Strings)
df_itensProva_silver = df_itensProva_silver.withColumn("area_de_conhecimento_do_item", F.col("area_de_conhecimento_do_item").cast(StringType()))
df_itensProva_silver = df_itensProva_silver.withColumn("gabarito_do_item", F.col("gabarito_do_item").cast(StringType()))
df_itensProva_silver = df_itensProva_silver.withColumn("motivo_para_o_abandono_do_item", F.col("motivo_para_o_abandono_do_item").cast(StringType()))
df_itensProva_silver = df_itensProva_silver.withColumn("cor_da_prova", F.col("cor_da_prova").cast(StringType()))
df_itensProva_silver = df_itensProva_silver.withColumn("lingua_estrangeira", F.col("lingua_estrangeira").cast(StringType()))

In [0]:
df_itensProva_silver.printSchema()

In [0]:
from pyspark.sql import functions as F

col_motivo = "motivo_para_o_abandono_do_item"
col_ind = "indicador_de_item_abandonado"
col_gab = "gabarito_do_item"

# 1 Corrige textos corrompidos vindos da BRONZE (substituições exatas)
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.when(F.col(col_motivo) == "Algoritmo NÃ£o convergiu", "Algoritmo não convergiu")
     .when(F.col(col_motivo) == "PedagÃ³gico", "Pedagógico")
     .otherwise(F.col(col_motivo))
)

# 2 Regra prioritária: se gabarito == 'X' -> anulado (indicador = 1 e motivo = Anulado)
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_ind,
    F.when(F.col(col_gab) == "X", F.lit(1)).otherwise(F.col(col_ind))
)

df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.when(F.col(col_gab) == "X", F.lit("Anulado no gabarito (X)")).otherwise(F.col(col_motivo))
)

# 3 Só marcar "Não aplicável" para casos claramente não-abandonado e sem motivo informado
# NÃO sobrescreve motivos já existentes ("Pedagógico") 
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.when(
        (F.col(col_ind) == 0) & 
        ( (F.col(col_motivo).isNull()) | (F.trim(F.col(col_motivo)) == "") ),
        F.lit("Não aplicável")
    ).otherwise(F.col(col_motivo))
)

# 4 Se indicador == 1 e motivo está nulo/vazio -> "Motivo não especificado"
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.when(
        (F.col(col_ind) == 1) &
        ( (F.col(col_motivo).isNull()) | (F.trim(F.col(col_motivo)) == "") ),
        F.lit("Motivo não especificado")
    ).otherwise(F.col(col_motivo))
)

# 5 Padronização final: mapear variações textuais para rótulos limpos, sem quebrar valores válidos
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.when(F.col(col_motivo).rlike("(?i)algoritmo.*converg"), "Algoritmo não convergiu")
     .when(F.col(col_motivo).rlike("(?i)pedag"), "Pedagógico")
     .when(F.col(col_motivo).rlike("(?i)anulad"), "Anulado no gabarito (X)")
     .when(F.col(col_motivo).rlike("(?i)motivo.*nao.*especific"), "Motivo não especificado")
     .when(F.col(col_motivo).rlike("(?i)nao aplic"), "Não aplicável")
     .otherwise(F.col(col_motivo))
)

# 6 Limpeza de espaços/virgulas duplicadas
df_itensProva_silver = df_itensProva_silver.withColumn(
    col_motivo,
    F.trim(F.regexp_replace(F.col(col_motivo), r"[\s,]+", " "))
)

# Verificação rápida: mostrar os rótulos finais e contagens
display(df_itensProva_silver.groupBy(col_motivo).count().orderBy(F.desc("count")))


In [0]:
display(df_itensProva_silver)

In [0]:
df_itensProva_silver = df_itensProva_silver.withColumn(
    "area_de_conhecimento_do_item",
    F.when(F.col("area_de_conhecimento_do_item") == "LC", "linguagens e codigos")
     .when(F.col("area_de_conhecimento_do_item") == "CH", "ciencias humanas")
     .when(F.col("area_de_conhecimento_do_item") == "CN", "ciencias da natureza")
     .when(F.col("area_de_conhecimento_do_item") == "MT", "matematica")
     .otherwise(F.col("area_de_conhecimento_do_item"))
)

In [0]:
df_itensProva_silver = df_itensProva_silver.withColumn(
    "lingua_estrangeira",
    F.when(
        F.col("area_de_conhecimento_do_item") != "linguagens e codigos",
        "não aplicavel"
    ).when(
        (F.col("area_de_conhecimento_do_item") == "linguagens e codigos") &
        F.col("lingua_estrangeira").isNull(),
        "indeterminado"
    ).when(
        F.col("lingua_estrangeira") == "0",
        "ingles"
    ).when(
        F.col("lingua_estrangeira") == "1",
        "espanhol"
    ).otherwise(F.col("lingua_estrangeira"))
)

In [0]:
display(df_itensProva_silver)

In [0]:
#Remover nulos das colunas (A, B, C)
df_itensProva_silver = df_itensProva_silver.withColumn(
    "parametro_de_discriminacao",
    F.when(F.col("parametro_de_discriminacao").isNull(), None)
     .otherwise(F.col("parametro_de_discriminacao"))
)

df_itensProva_silver = df_itensProva_silver.withColumn(
    "parametro_de_dificuldade",
    F.when(F.col("parametro_de_dificuldade").isNull(), None)
     .otherwise(F.col("parametro_de_dificuldade"))
)

df_itensProva_silver = df_itensProva_silver.withColumn(
    "parametro_de_acerto_ao_acaso",
    F.when(F.col("parametro_de_acerto_ao_acaso").isNull(), None)
     .otherwise(F.col("parametro_de_acerto_ao_acaso"))
)

In [0]:
display(df_itensProva_silver) 

In [0]:
# 1 padroniza tudo para minúsculo
df_itensProva_silver = df_itensProva_silver.withColumn(
    "cor_da_prova",
    F.lower(F.col("cor_da_prova"))
)

# 2 se prova adaptada → cor deve ser "leitor tela"
df_itensProva_silver = df_itensProva_silver.withColumn(
    "cor_da_prova",
    F.when(
        F.col("item_pertencente_a_prova_adaptada") == 1,
        F.lit("leitor tela")
    ).otherwise(F.col("cor_da_prova"))
)

# 3 se cor é "leitor tela" → força adaptada = 1
df_itensProva_silver = df_itensProva_silver.withColumn(
    "item_pertencente_a_prova_adaptada",
    F.when(
        F.col("cor_da_prova") == "leitor tela",
        F.lit(1)
    ).otherwise(F.col("item_pertencente_a_prova_adaptada"))
)

In [0]:
display(df_itensProva_silver)

In [0]:
df_itensProva_silver.write.format("delta").mode("overwrite").option(
    "overwriteSchema", "true"
).saveAsTable("setoreducacao.silver.itensProva_tratados_2024")
